# Computes Fragments from the PDB Fragment File

First select which data set you would like to generate.

In [ ]:
import ipywidgets as widgets
from glob import glob
from os.path import basename, splitext

datasets = [basename(x) for x in glob("Geometry/*")]
datadrop = widgets.Dropdown(
    options=datasets,
    description='Data Set:',
)
display(datadrop)

## Define the Input Files

In [ ]:
namelist = []
dataset = datadrop.value
for f in glob("Geometry/"+dataset+"/*.pdb"):
    namelist.append(splitext(basename(f))[0])

## Read the input

In [ ]:
from yaml import load
frag_indices = {}

fdicts = {}
for f in namelist:
    fdicts[f] = {}
    with open("Geometry/"+dataset+"/"+f+".pdb", 'r') as ifile:
        for line in ifile:
            split = line.split()
            if split[0] != "ATOM":
                continue
            atom = int(split[1])
            fname = split[3]
            fid = int(split[4])
            if fname not in fdicts[f]:
                fdicts[f][fname] = {}
            if fid not in fdicts[f][fname]:
                fdicts[f][fname][fid] = []
            fdicts[f][fname][fid].append(atom)

## Convert To A List of Lists

In [ ]:
frag_lists = {}
for f in fdicts:
    new_list = []
    slist = []
    for key, value in fdicts[f].items():
        for key2, value2 in fdicts[f][key].items():
            new_list.append(value2)
            slist.append(int(key2))
    frag_lists[f] = [x for _,x in sorted(zip(slist,new_list))]
#     frag_lists[f] = new_list

## Write To Yaml File

In [ ]:
import os
from os.path import exists
from os import makedirs

for f in namelist:
    with open("Geometry/"+dataset+"/"+f+"-frag.yaml", 'w') as ofile:
        for value in frag_lists[f]:
            ofile.write("- "+str(value)+"\n")